Sidharth Dhawan | 102218069 | BS3

Part I: Based on Feature Selection, Cleaning, and Preprocessing to Construct an Input from Data
Source

Import Libraries

In [61]:
import pandas as pd
import numpy as np

Load Dataset

In [62]:
customers = pd.read_csv('AWCustomers.csv')
sales = pd.read_csv('AWSales.csv')

print(customers.head())
print(sales.head())


   CustomerID Title FirstName MiddleName  LastName Suffix  \
0       21173   NaN      Chad          C      Yuan    NaN   
1       13249   NaN      Ryan        NaN     Perry    NaN   
2       29350   NaN     Julia        NaN  Thompson    NaN   
3       13503   NaN  Theodore        NaN     Gomez    NaN   
4       22803   NaN  Marshall          J      Shan    NaN   

             AddressLine1 AddressLine2         City    StateProvinceName  ...  \
0      7090 C. Mount Hood          NaN   Wollongong      New South Wales  ...   
1     3651 Willow Lake Rd          NaN      Shawnee     British Columbia  ...   
2  1774 Tice Valley Blvd.          NaN  West Covina           California  ...   
3         2103 Baldwin Dr          NaN    Liverpool              England  ...   
4         Am Gallberg 234          NaN        Werne  Nordrhein-Westfalen  ...   

         Education      Occupation Gender MaritalStatus HomeOwnerFlag  \
0        Bachelors        Clerical      M             M             1   


(a) Examine the values of each attribute and Select a set of attributes only that would affect to predict
future bike buyers to create your input for data mining algorithms. Remove all the unnecessary
attributes. (Select features just by analysis). 


Manually filtering data by dropping irrelevant columns

In [63]:
filtered_customers = customers[['CustomerID', 'PostalCode', 'Occupation', 'HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome', 'YearlyIncome']]

(b) Create a new Data Frame with the selected attributes only

In [64]:
merged_data = pd.merge(filtered_customers, sales, on='CustomerID', how='inner')

merged_data.to_csv('merged_data.csv', index=False)
merged_data.head()

,CustomerID,PostalCode,Occupation,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,YearlyIncome,BikeBuyer,AvgMonthSpend
0,21173,2500,Clerical,1,3,0,81916,1,50.97
1,13249,V9B 2C3,Clerical,1,2,1,81076,1,53.11
2,29350,91791,Clerical,0,3,0,86387,1,54.08
3,13503,L4 4HB,Skilled Manual,1,2,1,61481,1,56.93
4,22803,59368,Skilled Manual,1,1,0,51804,1,55.41


(c) Determine a Data value type (Discrete, or Continuous, then Nominal, Ordinal, Interval, Ratio) of
each attribute in your selection to identify preprocessing tasks to create input for your data mining. 

In [65]:
data3 = pd.read_csv('merged_data.csv')

print("Column data types:")
print(data3.dtypes)


data_types = {
    'CustomerID': 'Discrete (Nominal)',
    'Education': 'Categorical (Ordinal)',
    'Occupation': 'Categorical (Nominal)',
    'Gender': 'Binary (Nominal)',
    'MaritalStatus': 'Binary (Nominal)',
    'HomeOwnerFlag': 'Binary (Nominal)',
    'NumberCarsOwned': 'Discrete (Ordinal)',
    'NumberChildrenAtHome': 'Discrete (Ordinal)',
    'TotalChildren': 'Discrete (Ordinal)',
    'YearlyIncome': 'Continuous (Ratio)',
    'BikeBuyer': 'Binary (Nominal)',
    'AvgMonthSpend': 'Continuous (Ratio)'
}

for col, dtype in data_types.items():
    print(f"Attribute: {col}, Data Type: {dtype}")


Column data types:
CustomerID                int64
PostalCode               object
Occupation               object
HomeOwnerFlag             int64
NumberCarsOwned           int64
NumberChildrenAtHome      int64
YearlyIncome              int64
BikeBuyer                 int64
AvgMonthSpend           float64
dtype: object
Attribute: CustomerID, Data Type: Discrete (Nominal)
Attribute: Education, Data Type: Categorical (Ordinal)
Attribute: Occupation, Data Type: Categorical (Nominal)
Attribute: Gender, Data Type: Binary (Nominal)
Attribute: MaritalStatus, Data Type: Binary (Nominal)
Attribute: HomeOwnerFlag, Data Type: Binary (Nominal)
Attribute: NumberCarsOwned, Data Type: Discrete (Ordinal)
Attribute: NumberChildrenAtHome, Data Type: Discrete (Ordinal)
Attribute: TotalChildren, Data Type: Discrete (Ordinal)
Attribute: YearlyIncome, Data Type: Continuous (Ratio)
Attribute: BikeBuyer, Data Type: Binary (Nominal)
Attribute: AvgMonthSpend, Data Type: Continuous (Ratio)


Part II: Data Preprocessing and Transformation

Depending on the data type of each attribute, transform each object from your preprocessed data.
Use all the data rows (~= 18000 rows) with the selected features as input to apply all the tasks below, do
not perform each task on the smaller data set that you got from your random sampling result.

Importing necessary modules

In [66]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

(a) Handling Null values

In [67]:
# Categorical 
categorical_cols = ['Occupation']
imputer_mode = SimpleImputer(strategy='most_frequent')
merged_data[categorical_cols] = imputer_mode.fit_transform(merged_data[categorical_cols])

# Numerical 
numerical_cols = ['YearlyIncome', 'NumberCarsOwned', 'NumberChildrenAtHome']
imputer_mean = SimpleImputer(strategy='mean')
merged_data[numerical_cols] = imputer_mean.fit_transform(merged_data[numerical_cols])

# remaining NULL Values
print(merged_data.isnull().sum())


CustomerID              0
PostalCode              0
Occupation              0
HomeOwnerFlag           0
NumberCarsOwned         0
NumberChildrenAtHome    0
YearlyIncome            0
BikeBuyer               0
AvgMonthSpend           0
dtype: int64


(b) Normalization

In [68]:
# Normalize YearlyIncome and AvgMonthSpend
scaler = MinMaxScaler()
merged_data[['YearlyIncome', 'AvgMonthSpend']] = scaler.fit_transform(merged_data[['YearlyIncome', 'AvgMonthSpend']])

# Display normalized columns
print(merged_data[['YearlyIncome', 'AvgMonthSpend']].head())

   YearlyIncome  AvgMonthSpend
0      0.496842       0.324210
1      0.489453       0.425201
2      0.536172       0.470977
3      0.317083       0.605474
4      0.231958       0.533742


(c) Discretization (Binning) on Continuous attributes or Categorical Attributes with too many different values

In [69]:
# Converting the continuous attribute (Yearly Income) to Discrete -> Low, Medium, High 
merged_data['IncomeBin'] = pd.cut(merged_data['YearlyIncome'], bins=3, labels=['Low', 'Medium', 'High'])

# binned data
print(merged_data[['YearlyIncome', 'IncomeBin']].head())

# Check the number of occurrences in each bin
income_bin_counts = merged_data['IncomeBin'].value_counts()

print(income_bin_counts)

   YearlyIncome IncomeBin
0      0.496842    Medium
1      0.489453    Medium
2      0.536172    Medium
3      0.317083       Low
4      0.231958       Low
IncomeBin
Low       9436
Medium    4464
High      4461
Name: count, dtype: int64


(d) Standardization/Normalization

In [70]:
# Normalizing YearlyIncome and AvgMonthSpend
scaler = StandardScaler()
merged_data[['YearlyIncome', 'AvgMonthSpend']] = scaler.fit_transform(merged_data[['YearlyIncome', 'AvgMonthSpend']])

print(merged_data[['YearlyIncome', 'AvgMonthSpend']].head())

   YearlyIncome  AvgMonthSpend
0      0.298555      -0.231774
1      0.271180       0.390755
2      0.444261       0.672929
3     -0.367401       1.501999
4     -0.682765       1.059828


(e) Binarization (One Hot Encoding) 

In [71]:
# Convert non-numeric data to numeric using one-hot encoding
merged_data = pd.get_dummies(merged_data)

# Ensure all values are numeric
merged_data = merged_data.apply(pd.to_numeric)


Part III: Calculating Proximity / Correlation Analysis of two features.

Make sure each attribute is transformed in a same scale for numeric attributes and Binarization for each
nominal attribute, and each discretized numeric attribute to standardization. Make sure to apply a correct
similarity measure for nominal (one hot encoding)/binary attributes and numeric attributes respectively. 

(a) Calculate Similarity in Simple Matching, Jaccard Similarity, and Cosine Similarity between two following objects of your transformed input data.

In [72]:
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import cosine

# Simple Matching Coefficient (SMC)
smc = sum(merged_data.iloc[0] == merged_data.iloc[1]) / len(merged_data.iloc[0])
print("Simple Matching Coefficient:", smc)

# Jaccard Similarity (for binary features only)
# Convert DataFrame row values to NumPy arrays for Jaccard
row1, row2 = merged_data.iloc[0].values, merged_data.iloc[1].values

# Ensure binary values (0/1) for Jaccard similarity
row1_binary = np.where(row1 > 0, 1, 0)
row2_binary = np.where(row2 > 0, 1, 0)

jaccard_sim = jaccard_score(row1_binary, row2_binary, average='macro')
print("Jaccard Similarity:", jaccard_sim)

# Cosine Similarity
cosine_sim = 1 - cosine(row1, row2)
print("Cosine Similarity:", cosine_sim)

Simple Matching Coefficient: 0.9792899408284024
Jaccard Similarity: 0.8121395221093106
Cosine Similarity: 0.999999990711113


(b) Calculate Correlation between two features Commute Distance and Yearly Income 

In [73]:
correlation = merged_data['AvgMonthSpend'].corr(merged_data['YearlyIncome'])
print("Correlation: ", correlation)

Correlation:  0.5301257155563448
